In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import numpy as np
import pandas as pd
from skimage.io import imread
from skimage.color import gray2rgb
import cv2
from pandas import read_csv
import matplotlib.pyplot as plt

%matplotlib inline

## Visualization

In [ ]:
from evaluate import plot_learning_curves, load_network
import cPickle as pickle
from models import nnet_three_conv_layer
from utils import get_file_list, create_fixed_image_shape
from nolearn.lasagne import BatchIterator
from plotting import plot_face_bb
from lazy_batch_iterator import LazyBatchIterator
import skimage
import glob

In [ ]:
train_folder = 'train/'
fnames, bboxes = get_file_list(train_folder)

In [ ]:
def prepare_test_img(shape, img=None, name=None):
    from skimage.io import imread 
    im = None
    if img is not None:
        im = img
    elif name is not None:
        im = imread(name)
    inp = create_fixed_image_shape(im, shape, mode='fit')
    return inp

In [ ]:
model_name = '/home/arya_03/work/face_detection_gtx/model_vgg_like_2/model_100.pkl'
full_model_name = '/home/arya_03/work/face_detection_gtx/model_vgg_like_2/full_nnet.pkl'

In [ ]:
nnet = load_network(model_name)

In [ ]:
nnet = pickle.load(open(full_model_name, 'rb'))

In [ ]:
nnet.batch_iterator_test = BatchIterator(batch_size=64)

In [ ]:
# Predict and plot the bounding boxes for images in test_imgs/ folder
for fname in glob.glob(os.path.join('test_imgs', '*.jpg')):
    proc_img = prepare_test_img((256, 256, 3), name=fname)
    img = np.transpose(proc_img, [2, 0, 1])
    preds = nnet.predict([img]) * .5 + .5
    plot_face_bb(proc_img, preds[0], path=False)

In [ ]:
idx = 1090
# nnet.batch_iterator_test = LazyBatchIterator(batch_size=64)
preds = nnet.predict([fnames[idx]]) * .5 + .5
plot_face_bb(fnames[idx], preds[0])
plot_face_bb(fnames[idx], np.asarray(bboxes[idx]) * .5 + .5)

## Augmentation

In [ ]:
from utils import get_file_list
import glob
from train_val_split import get_names_bboxes

In [ ]:
folder = '/media/shared/faceScrub/train_face_det/'
path = folder
actor_label_txt = '/media/shared/faceScrub/facescrub_actors.txt'
actress_label_txt = '/media/shared/faceScrub/facescrub_actresses.txt'
accept_pattern = '*/*.jpg'

In [ ]:
# def get_names_bboxes1(folder, actor_label_txt, actress_label_txt, accept_pattern='*.jpg'):
full_names = glob.glob(os.path.join(folder, accept_pattern))
only_names = map(lambda f: os.path.splitext(
    os.path.basename(f))[0], full_names)

pd_male = read_csv(actor_label_txt, sep='\t')
del pd_male['url'], pd_male['image_id'], pd_male['face_id']
pd_female = read_csv(actress_label_txt, sep='\t')
del pd_female['url'], pd_female['image_id'], pd_female['face_id']
pd_celeb = pd.concat([pd_male, pd_female], ignore_index=True)
pd_celeb = pd_celeb.drop_duplicates(subset='sha256', keep='last').set_index('sha256')

bboxes = map(lambda k: [np.float32(v) for v in 
    pd_celeb.bbox[k].split(',')], only_names)
# return full_names, bboxes

In [ ]:
fnames, bboxes = get_names_bboxes1(path, actor_label_txt, actress_label_txt, accept_pattern='*/*.jpg')

In [ ]:
cc = globals()['get_names_bboxes1']

In [ ]:
cc()

In [ ]:
len(fnames), len(bboxes)

In [ ]:
pd_celeb.ix[only_names[-10:]].to_csv('try.csv', sep='\t', index_label='sha256')

In [ ]:
df = pd.DataFrame({'name': fnames, 'bbox': bboxes})
df.head()

In [ ]:
df.bbox[[1, 4]].to_dense()

In [ ]:
!head try.csv

## Net construction

In [ ]:
from fd import get_file_list, nnet_three_conv_layer
from plotting import plot_face_bb

In [ ]:
nnet = nnet_three_conv_layer()

In [ ]:
train_folder = 'train/'

In [ ]:
fnames, bboxes = get_file_list(train_folder)

In [ ]:
fnames[0], bboxes[0]

In [ ]:
X = np.array(list(fnames))
y = np.array(list(bboxes))

In [ ]:
nnet.fit(X, y)

In [ ]:
plot_face_bb(fnames[1000], bboxes[1000])

In [ ]:
im = imread(fnames[0])

In [ ]:
X = []
X.append(np.transpose(im, [2,0,1]))
X.append(None)
X.append(np.transpose(im, [2,0,1]))
X.append(None)

In [ ]:
np.array(X).shape

In [ ]:
plt.imshow(imread('/home/arya_03/data/concat/data/0/bb008b9e367f001837b3adabb2b46323.jpg'))